In [ ]:
# be sure to have the requirements
!pip install requirements

In [ ]:
import pandas as pd

import seaborn as sns
import plotly.express as px
sns.set(rc={'figure.figsize':(20,6)})

# main functions
from outlier_detection import detect_outliers, rolling_outlier_detector

#### Loading toy dataset
Should be time sorted

In [7]:
# loading the dataset
df = pd.read_csv("example_dataset/trilux_dataset.csv")
# filtering
df = (
    df
    .loc[df["trabajos_id"]==1]
    .filter(items=["fecha", "chla", "tby", "pc"])
    .set_index("fecha")
    .sort_index()
)
df

,chla,tby,pc
fecha,,,
2021-07-02 17:30:37.000,3.681,0.085,0.380
2021-07-02 17:30:58.000,4.216,0.096,0.510
2021-07-02 17:31:20.000,3.453,-0.042,0.512
2021-07-02 17:31:42.000,3.315,0.113,0.377
2021-07-02 17:32:03.000,4.367,-0.040,0.332
...,...,...,...
2021-12-15 17:53:26.000,8.114,23.992,1.633
2021-12-15 17:53:48.000,8.497,23.885,1.619
2021-12-15 17:54:10.000,8.348,23.979,1.524


#### Detecting a single sample 

In [158]:
detect_outliers?

Signature:
detect_outliers(
    past_data: pandas.core.frame.DataFrame,
    current_data: pandas.core.frame.DataFrame,
    elasticity: int,
) -> int
Docstring:
Classifies if 'current_data' is an outlier or not, given 'past_data'.

Args:
    past_data: a sorted time indexed dataframe (ascending order) all collumns
        are considered into the outlier detection.
    current_data: a single row dataframe to evaluate if is an outlier or not, 
        'current_data' should have the same columns as 'past_data' and corresponds
        to the closest next value from 'past_data'.
    elasticity: integer from 1 to 3 that determines the level of elasticity of 
        outlier detection algorithm, being 1 the less elastic and 3 the more flexible.

Returns:
    -1 for outliers, 1 for inliers
    
File:      ~/Documents/gits/timeseries_outlier_detection/outlier_detection.py
Type:      function


In [154]:
# first 100 samples as past data
past_data = df.iloc[:100]
# next sample as current data
current_data = df.iloc[101].to_frame().T

# detect if is outlier with flexibility 3 (a relaxed approach)
detect_outliers(past_data, current_data, 3)

1

returns 1, which means that sample #101 isn't an outlier (based on the previous 100 samples)

#### Detecting outliers on a dataset, with a rolling window

In [157]:
# select 10k samples for time efficiency
N_SAMPLES = 10000 # let's perform outlier detection on 10k samples
WINDOW_SIZE = 100 # analyzed 100 samples in order to classify the next
ELASTICITY = 3 # relaxed detector

mini_df = df.iloc[:N_SAMPLES]
results = rolling_outlier_detector(mini_df, WINDOW_SIZE, ELASTICITY)


100%|██████████| 10000/10000 [09:30<00:00, 17.54it/s]


#### Plotting results

In [151]:
px.scatter(mini_df.reset_index(), x="fecha" , y="chla", color=results)
